In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import re
import nltk
from nltk.corpus import stopwords
import numpy as np
from sklearn.decomposition import PCA
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')

Pickle savings helpers

In [ ]:

import pickle

def save_data(object_name, object):
    """
    Save the given object to a pickle file.

    Parameters:
    object_name (str): The name of the object.
    object: The object to be saved.
    """
    with open(f'./data/pickles/{object_name}.pickle', 'wb') as f:
        pickle.dump(object, f)

def load_data(object_name, calculator, *args):
    """
    Load the data from a pickle file. If the file doesn't exist, calculate the data using the given calculator function,
    save it to a pickle file, and return the calculated data.

    Parameters:
    object_name (str): The name of the object.
    calculator (function): The function to calculate the data if the file doesn't exist.
    *args: Additional arguments to be passed to the calculator function.

    Returns:
    The loaded or calculated data.
    """
    try:
        with open(f'./data/pickles/{object_name}.pickle', 'rb') as f:
            return pickle.load(f)
    except FileNotFoundError:
        obj = calculator(*args)
        save_data(object_name, obj)
        return obj

In [ ]:
# data loading
PATH = './data/ruddit.csv'
TRAIN_RATIO = 0.75
TEST_VAL_RATIO = 1

dataset = pd.read_csv(PATH)
x_train, x_test_valid, y_train, y_test_valid = train_test_split(dataset["comment_text"], dataset['offensiveness_score'] , train_size=TRAIN_RATIO, random_state=0)
x_test, x_valid, y_test, y_valid = train_test_split(x_test_valid, y_test_valid, test_size=TEST_VAL_RATIO, random_state=0)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_test = x_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)
train_raw = pd.DataFrame({'text': x_train, 'score': y_train})
test_raw = pd.DataFrame({'text': x_test, 'score': y_test})
valid_raw = pd.DataFrame({'text': x_valid, 'score': y_valid})
train_raw['score'] = train_raw['score'].astype('float32')
test_raw['score'] = test_raw['score'].astype('float32')
valid_raw['score'] = valid_raw['score'].astype('float32')
del x_train, x_test, x_valid, y_train, y_test, y_valid, x_test_valid, y_test_valid
len(train_raw), len(test_raw), len(valid_raw)

Dataset Cleaning

In [ ]:
wl = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    pattern = re.compile('[^a-zA-Z]')
    text = pattern.sub(' ', text)
    text = text.lower()
    text = text.split()
    text = [wl.lemmatize(word) for word in text if not word in stop_words]
    text = ' '.join(text)
    return text

def clean_dataset(ds):
    ds['text'] = ds['text'].apply(preprocess_text)
    return ds

train = clean_dataset(train_raw.copy())
test = clean_dataset(test_raw.copy())
valid = clean_dataset(valid_raw.copy())

Bag of words representations

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
pca = PCA(n_components=300)
bow_train = vectorizer.fit_transform(train_raw['text']).toarray()
bow_test = vectorizer.fit_transform(test_raw['text']).toarray()
bow_valid = vectorizer.fit_transform(valid_raw['text']).toarray()

pad_width = ((0, 0), (0, bow_train.shape[1] - bow_test.shape[1])) 
bow_test = np.pad(bow_test, pad_width, mode='constant', constant_values=0)
pad_width = ((0, 0), (0, bow_train.shape[1] - bow_valid.shape[1])) 
bow_valid = np.pad(bow_valid, pad_width, mode='constant', constant_values=0)

bow_train = pca.fit_transform(bow_train)
bow_test = pca.fit_transform(bow_test)
# bow_valid = pca.fit_transform(bow_valid)


bow_train.shape, bow_test.shape, bow_valid.shape

In [ ]:
bow_train.shape

TFIDF representations

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
tfidf_train = vectorizer.fit_transform(train_raw['text']).toarray()
tfidf_test = vectorizer.fit_transform(test_raw['text']).toarray()
tfidf_valid = vectorizer.fit_transform(valid_raw['text']).toarray()

pad_width = ((0, 0), (0, tfidf_train.shape[1] - tfidf_test.shape[1])) 
tfidf_test = np.pad(tfidf_test, pad_width, mode='constant', constant_values=0)
pad_width = ((0, 0), (0, tfidf_train.shape[1] - tfidf_valid.shape[1])) 
tfidf_valid = np.pad(tfidf_valid, pad_width, mode='constant', constant_values=0)

tfidf_train = pca.fit_transform(tfidf_train)
tfidf_test = pca.fit_transform(tfidf_test)

tfidf_train.shape, tfidf_test.shape, tfidf_valid.shape

Embeddings representations

In [ ]:
from gensim.models import Word2Vec
import gensim.downloader as api 

w2vec_google_news_model = load_data('w2vec_google_news_model', api.load, 'word2vec-google-news-300')
glove_twitter_model = load_data('glove_twitter_model', api.load, 'glove-wiki-gigaword-300')
fasttext_wiki_news_model = load_data('fasttext_wiki_news_model', api.load, 'fasttext-wiki-news-subwords-300')


In [ ]:
tokenized_train = [text.split() for text in train_raw['text']]
tokenized_test = [text.split() for text in test_raw['text']]

In [ ]:
def vectorize(sentence_tokens, w2v_model, vector_size=300):
    words_vecs = [w2v_model[word] for word in sentence_tokens if word in w2v_model]
    if len(words_vecs) == 0:
        return np.zeros(vector_size)
    words_vecs = np.array(words_vecs)
    return words_vecs.mean(axis=0)

train_fasttext = ([vectorize(tokens, fasttext_wiki_news_model) for tokens in tokenized_train])
test_fasttext = ([vectorize(tokens, fasttext_wiki_news_model) for tokens in tokenized_test])
train_v2w = ([vectorize(tokens, w2vec_google_news_model) for tokens in tokenized_train])
test_v2w = ([vectorize(tokens, w2vec_google_news_model) for tokens in tokenized_test])
train_glove = ([vectorize(tokens, glove_twitter_model) for tokens in tokenized_train])
test_glove = ([vectorize(tokens, glove_twitter_model) for tokens in tokenized_test])


LLMs representations

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

In [ ]:
import torch

def bert_embeddings(ds):
    # Tokenize the texts
    tokenized_inputs = tokenizer(list(ds), truncation=True, padding=True, return_tensors="pt").to(device)
    tokenized_inputs.to('cuda')
    # Generate BERT embeddings
    with torch.no_grad():
        model_output = model(**tokenized_inputs)

    # Extract the embeddings
    embeddings = model_output.last_hidden_state
    embeddings = embeddings.cpu()

    torch.cuda.empty_cache()
    
    return embeddings.mean(dim=1).numpy()

In [ ]:
def bert_embeddings_cuda(ds):
    batch_size = 100  # You can adjust this according to your GPU memory capacity

    # Initialize a list to store embeddings for each text
    embeddings_list = []
    final_embeddings = torch.tensor([])
    # Process texts in batches
    for i in range(0, len(ds), batch_size):
        batch_texts = ds[i:i+batch_size]
        batch_tokens = tokenizer(batch_texts.tolist(), return_tensors='pt', padding=True, truncation=True) 
        batch_tokens.to(device)        
        with torch.no_grad():
            outputs = model(**batch_tokens)
        last_hidden_states = outputs.last_hidden_state

        last_hidden_states = last_hidden_states.cpu().numpy()
        embeddings = np.mean(last_hidden_states, axis=1)
        final_embeddings = torch.cat((final_embeddings, torch.tensor(embeddings)), 0)
        # Append the embeddings to the list
        # embeddings_list.append(last_hidden_states)
        
        # Empty CUDA memory
        torch.cuda.empty_cache()

    return final_embeddings.numpy()
    # Convert the embeddings list to a numpy array
    # embeddings_array = np.concatenate(embeddings_list, axis=0)
    # return torch.tensor(embeddings_array).numpy()

In [ ]:
train_bert = bert_embeddings_cuda(train_raw['text'])
test_bert = bert_embeddings_cuda(test_raw['text'])

In [ ]:
train_bert.shape, test_bert.shape

In [ ]:
# train_bert = bert_embeddings(train['text'])
# test_bert = bert_embeddings(test['text'])

In [ ]:
datasets = {
    'bow': (bow_train, bow_test, bow_valid),
    'tfidf': (tfidf_train, tfidf_test, tfidf_valid),
    'fasttext': (train_fasttext, test_fasttext),
    'word2vec': (train_v2w, test_v2w),
    'glove': (train_glove, test_glove),
    'bert': (train_bert, test_bert)
}

In [ ]:
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor

linear_reg = LinearRegression()
svr_reg = SVR(kernel = 'rbf')
mlp_reg = MLPRegressor(random_state=1, max_iter=500)
rf_reg = RandomForestRegressor(n_estimators=100, random_state=42)


In [ ]:
model_train = tfidf_train
model_test = tfidf_test

In [ ]:
linear_reg.fit(model_train, train['score'])
svr_reg.fit(model_train, train['score'])
mlp_reg.fit(model_train, train['score'])
rf_reg.fit(model_train, train['score'])

In [ ]:
svr_preds = svr_reg.predict(model_test)
linear_preds = linear_reg.predict(model_test)
mlp_preds = mlp_reg.predict(model_test)
rf_preds = rf_reg.predict(model_test)

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, classification_report
svr_mse = mean_squared_error(test['score'], svr_preds)
linear_mse = mean_squared_error(test['score'], linear_preds)
mlp_mse = mean_squared_error(test['score'], mlp_preds)
rf_mse = mean_squared_error(test['score'], rf_preds)
print(f'SVR MSE: {svr_mse}, Linear MSE: {linear_mse}, MLP MSE: {mlp_mse}, RF MSE: {rf_mse}')

In [ ]:
times_clean = {
    'fasttext': 50.6,
    'word2vec': 48.9,
    'glove': 49.4,
    'bert_embeddings': 113
}
times_dirty = {
    'fasttext': 55.7,
    'word2vec': 54.3,
    'glove': 54.2,
    'bert_embeddings': 117
}
results_mse_cleaned = {
    'bow': {'SVR': 0.12376995930427524, 'Linear': 0.17338464620420832, 'MLP': 0.2564379379779361, 'RF': 0.1131010225230955},
    'tfidf': {'SVR': 0.12007776170149795, 'Linear': 0.1410922185815179, 'MLP': 0.22373554702562642, 'RF': 0.10625150627656584},
    'fasttext': {'SVR': 0.04685816665354699, 'Linear': 0.07273326632161381, 'MLP': 0.05643423252877395, 'RF': 0.06449617951793207},
    'Word2Vec': {'SVR': 0.04816666309223599, 'Linear': 0.07068408441559317, 'MLP': 0.0655118452881107, 'RF': 0.07014625622457209},
    'Glove': {'SVR': 0.05030112790450738, 'Linear': 0.0782962824967885, 'MLP': 0.07334035831215925, 'RF': 0.07286072318841706},
    'bert_embeddings': {'SVR': 0.05263384180285866, 'Linear': 0.058188606053590775, 'MLP': 0.07364223152399063, 'RF': 0.06752015332228646}
}
results_mse_dirty = {
    'bow': {'SVR': 0.12816722954426424, 'Linear': 0.1646148816792815, 'MLP': 0.21052087127847655, 'RF': 0.11326946991419015},
    'tfidf': {'SVR': 0.13030425877084306, 'Linear': 0.17210081402548744, 'MLP': 0.23246958937586945, 'RF': 0.11589168438060171,},
    'fasttext': {'SVR': 0.058821879162422326, 'Linear': 0.08009510252213108, 'MLP': 0.08096923470973573, 'RF': 0.07731245249116338},
    'Word2Vec': {'SVR': 0.05957748697334124, 'Linear': 0.08946030570644542, 'MLP': 0.09698301239891266, 'RF': 0.08042184426460584},
    'Glove': {'SVR': 0.07340861310598293, 'Linear': 0.09466922426057676, 'MLP': 0.10379193202544125, 'RF': 0.08534953326234156},
    'bert_embeddings':{'SVR': 0.04606831641743619, 'Linear': 0.05261010304093361, 'MLP': 0.06015215069055557, 'RF': 0.0667452847313885}
}

In [ ]:
df_cleaned = pd.DataFrame(results_mse_cleaned)
df_dirty = pd.DataFrame(results_mse_dirty)

In [ ]:
with pd.ExcelWriter('output.xlsx') as writer:
    df_cleaned.to_excel(writer, sheet_name='Results_Cleaned')
    df_dirty.to_excel(writer, sheet_name='Results_Dirty')